In [1]:
from utils import *
from prompt_generators import create_basic_prompt, generate_coding_prompt, generate_prompt_with_image, generate_code_fixing_prompt, create_reasoning_model_prompt
from llm_utils import generate_llm_response, initialize_client
from eval import extract_solution, evaluate_solution, convert_output_to_array, extract_code, execute_transform_function

import os

import ast
import time
import json
import asyncio

import numpy as np
import pandas as pd
     
config = {
    'model': 'pixtral-large-latest',
    'solution_type': 'simple'
}

client = initialize_client('mistral')
print("Client initialized!")

ModuleNotFoundError: No module named 'utils'

In [ ]:
# Create test_set
check_dataset()
train, eval = load_files_from_json()
test_set = filter_tasks_by_grid_size(train)
print(f"Test set has {len(test_set)} tasks in it.")

ARC data complete.
Test set has 300 tasks in it.


In [1]:
#######################################
#   Used this for oneshot simple prompt
#######################################
results = []

for i, (task_name, task_content) in enumerate(test_set.items()):

    print(f"Processing task {task_name}")
    print(f"#{i +1} out of {len(test_set)}")
    
    messages = create_basic_prompt(task=task_content, allow_reasoning=False)
    
    print("Asking llm for answer!")
    response = generate_llm_response(messages=messages, 
                                     client=client,
                                     model=config['model'],
                                     num_samples=1,
                                     temperature=0.1)
    
    print("Evaluating answer")
    
    extracted_answer = None     # Initialize to None before the try block
    try:
        extracted_answer_temp = extract_solution(response)            
        extracted_answer = convert_output_to_array(extracted_answer_temp)
    except ValueError as e:
        print(f"Error converting extracted solution to NumPy array: {e}")
        extracted_answer = None
    
    correct_solution = np.array(task_content['test'][0]['output'])
    result = evaluate_solution(extracted_answer, correct_solution)
    
    print(result)
    
    row = (
        {
            'task': task_name,
            'llm_full_answer': response,
            'llm_extracted_answer': extracted_answer
        })
    
    row.update(result)
    results.append(row)
    
    # to not exceed usage rates
    print("Waiting for one second...")
    time.sleep(1)
    
    
    df = pd.DataFrame(results)
    df.to_csv("results/vl_models/Pixtral-Large-123B_baseline.csv", index=False)
    print(f"Data saved to CSV after {i + 1} iterations.")
 
df = pd.DataFrame(results)
df.to_csv("results/vl_models/Pixtral-Large-123B_baseline.csv", index=False)
print("Data saved to CSV at the end of the loop.")

NameError: name 'test_set' is not defined

In [ ]:
df.query("percentage_correct == 1")

task                    36
llm_full_answer         36
llm_extracted_answer    36
answer_extracted        36
correct_grid_size       36
percentage_correct      36
dtype: int64